In [1]:
from heapq import heappop, heappush

import numpy as np

In [4]:
class Node:
    def __init__(self, level, x0, x1, num_var, 
               branch_strategy, subgrad_strategy):
        self.level = level
        self.x0 = x0
        self.x1 = x1
        self.num_var = num_var
        self.branch_strategy = branch_strategy
        self.subgrad_strategy = subgrad_strategy
        self.lb = None
        self.x_lb = None
        self.lambd = None

    def generate_children(self, A, b):
        return self.branch_strategy(A, b, self)

    def is_leaf(self):
        # True if all variables have been fixed
        return len(self.x0) + len(self.x1) == self.num_var
    
    def compute_lb(self, A, b, ub):
        self.lambd, self.lb, self.x_lb = \
            self.subgrad_strategy(A, b, ub, self.x0, self.x1)
        
    def get_val(self, A):
        x = np.array((self.num_var,))
        x[self.x0] = 0
        x[self.x1] = 1
        return np.sum(A, axis=0) @ x
    
    def get_lb(self):
        return self.lb
    
    def get_x_lb(self):
        return self.x_lb
    
    def get_lambd(self):
        return self.lambd
    
    def get_x0(self):
        return self.x0
    
    def get_x1(self):
        return self.x1
    
    def get_num_var(self):
        return self.num_var
    
    def get_subgrad_strategy(self):
        return self.subgrad_strategy

In [5]:
n = Node()

TypeError: Node.__init__() missing 6 required positional arguments: 'level', 'x0', 'x1', 'num_var', 'branch_strategy', and 'subgrad_strategy'

In [ ]:
class Tree:
    def _init_(self, root):
        self.open_list = [root]

    def is_empty(self):
        return len(self.open_list) == 0
    
    def remove(self):
        heappop(self.open_list)
    
    def add(self, node):
        heappush(self.open_list, (node.get_lb(), node))

In [ ]:
class BranchAndBound:
    def _init_(self, process_root_f=None):
        self.tree = Tree(Node(0, 0))
        self.best = None
        self.ub = np.inf
        self.process_root_f = process_root_f

    def search(self):
        while (not self.tree.is_empty()):
            father = self.tree.remove()
            if father.get_level() == 0:
                self.process_root(father)
                father.compute_lb()
            self.branch(father)
            
    def process_root(self, root):
        if self.process_root_f is not None:
            self.process_root_f(root)

    def branch(self, father):
        for child in father.generate_children():
            if child.is_leaf():
                self.evaluate_leaf(child)
            else:
                child.compute_lb()
                if child.get_lb() < self.ub:   
                    self.tree.add(child)
            
    def evaluate_leaf(self, leaf):
        leaf_val = leaf.get_val()
        if (self.best is None) or \
            (leaf_val < self.best.get_val()):
            self.best = leaf
            self.ub = leaf_val

In [ ]:
def branch_strategy(A, b, node):
    x0 = node.get_x0()
    x1 = node.get_x1()

    # Pick the row with the largest violation given the
    # solution obtained from Lagrangean relaxation
    r = np.argmax(b - A @ node.get_x_lb())
    
    # Columns with fixed values or with a zero entry
    # on the picked row are not candidates for branching
    zero_entry = set(np.where(A[r] == 0)[0])
    not_candidates = list(set(x0).union(set(x1)).union(zero_entry))
    
    # Pick the column with minimum reduced cost
    rc = (1 - node.get_lambd()) @ A
    rc[not_candidates] = np.inf
    j = np.argmin(rc)

    return [
        Node(node.get_level() + 1, x0 + [j], x1, 
             node.get_num_var(), branch_strategy, 
             node.get_subgrad_strategy()),
        Node(node.get_level() + 1, x0, x1 + [j], 
             node.get_num_var(), branch_strategy, 
             node.get_subgrad_strategy()),     
    ]